In [1]:
import numpy as np 
import matplotlib.pyplot as plt

from sklearn import metrics
import seaborn as sns

from struct import unpack
import datetime
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier

In [2]:
def loadMnist(imagefile, labelfile):
    images = open(imagefile, 'rb')
    labels = open(labelfile, 'rb')
    images.read(4)
    number_of_images = images.read(4)
    number_of_images = unpack('>I', number_of_images)[0]
    rows = images.read(4)
    rows = unpack('>I', rows)[0]
    cols = images.read(4)
    cols = unpack('>I', cols)[0]
    labels.read(4)
    N = labels.read(4)
    N = unpack('>I', N)[0]
    x = np.zeros((N, rows*cols), dtype=np.uint8)  # Initialize numpy array
    y = np.zeros(N, dtype=np.uint8)  # Initialize numpy array
    for i in range(N):
        for j in range(rows*cols):
            tmp_pixel = images.read(1)  # Just a single byte
            tmp_pixel = unpack('>B', tmp_pixel)[0]
            x[i][j] = tmp_pixel
        tmp_label = labels.read(1)
        y[i] = unpack('>B', tmp_label)[0]
    images.close()
    labels.close()
    return (x, y)

In [3]:
TrainImg, TrainLb = loadMnist('data/train-images-idx3-ubyte'
                                 , 'data/train-labels-idx1-ubyte')
TestImg, TestLb = loadMnist('data/t10k-images-idx3-ubyte'
                               , 'data/t10k-labels-idx1-ubyte')

In [4]:
TrainImg2 = TrainImg.reshape(60000,28,28)
TestImg2 = TestImg.reshape(10000,28,28)

In [5]:
 def dataCropping(CropData,Length):
    CropDataFinsh = np.empty([Length,20,20])
    for i in range(Length):
        Row = np.delete(CropData[i],[0,1,2,3,24,25,26,27],axis=0)
        Column = np.delete(Row,[0,1,2,3,24,25,26,27],axis=1)
        CropDataFinsh[i] = Column
    return CropDataFinsh

In [7]:
TrainImgCrop2 = dataCropping(TrainImg2,60000)
TestImgCrop2 = dataCropping(TestImg2,10000)
TrainImgCrop = TrainImgCrop2.reshape(60000,400)
TestImgCrop = TestImgCrop2.reshape(10000,400)

In [8]:
TrainImgCrop2Tr, TrainImgCrop2Te, TrainLbTr, TrainLbTe = train_test_split(TrainImgCrop2, TrainLb, train_size=0.9)
TrainImgCropNor2Tr = TrainImgCrop2Tr/255
TrainImgCropNor2Te = TrainImgCrop2Te/255
TrainImgCropNor2 = TrainImgCrop2/255
TestImgCropNor2 = TestImgCrop2/255

In [9]:
BestModel = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[20, 20]),
    keras.layers.Dense(400, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
opt = keras.optimizers.SGD(learning_rate=0.4)
BestModel.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [14]:
starttime = datetime.datetime.now()
BestModel.fit(TrainImgCropNor2Tr, TrainLbTr, epochs=40,validation_data=(TrainImgCropNor2Te, TrainLbTe))
endtime = datetime.datetime.now()
print("the time of Best Model is {}".format( endtime - starttime))

Epoch 1/40
1688/1688 [==============================] - 1s 837us/step - loss: 0.0042 - accuracy: 0.9987 - val_loss: 0.1401 - val_accuracy: 0.9795
Epoch 2/40
1688/1688 [==============================] - 1s 793us/step - loss: 0.0065 - accuracy: 0.9983 - val_loss: 0.1647 - val_accuracy: 0.9787
Epoch 3/40
1688/1688 [==============================] - 1s 795us/step - loss: 0.0094 - accuracy: 0.9974 - val_loss: 0.1620 - val_accuracy: 0.9793
Epoch 4/40
1688/1688 [==============================] - 1s 793us/step - loss: 0.0128 - accuracy: 0.9965 - val_loss: 0.1701 - val_accuracy: 0.9748
Epoch 5/40
1688/1688 [==============================] - 1s 792us/step - loss: 0.0142 - accuracy: 0.9962 - val_loss: 0.1561 - val_accuracy: 0.9770
Epoch 6/40
1688/1688 [==============================] - 1s 792us/step - loss: 0.0081 - accuracy: 0.9976 - val_loss: 0.1485 - val_accuracy: 0.9788
Epoch 7/40
1688/1688 [==============================] - 1s 793us/step - loss: 0.0157 - accuracy: 0.9960 - val_loss: 0.2046 -

In [15]:
BestPreDou = BestModel.predict(TestImgCropNor2)

In [16]:
BestPre = np.empty([10000])
for i in range(10000):
    x = 0
    for j in range(10):
        if(x < BestPreDou[i][j]):
            x = BestPreDou[i][j]
            BestPre[i] = j

In [21]:
BestAcc = accuracy_score(BestPre,TestLb)
print("the best accuracy is {}".format(BestAcc))
BestRec = recall_score(BestPre,TestLb,average='macro')
print("the best recall is {}".format(BestRec))
BestPreci = precision_score(BestPre,TestLb,average='macro')
print("the best precision is {}".format(BestPreci))
BestConMat = confusion_matrix(BestPre,TestLb)
print(BestConMat)

the best accuracy is 0.9848
the best recall is 0.984755984639501
the best precision is 0.9846442874137296
[[ 976    0    2    1    1    2    5    1    1    2]
 [   1 1129    1    0    1    0    2    2    0    2]
 [   0    1 1014    1    2    0    1    7    3    0]
 [   0    1    2  996    0    6    1    1    3    4]
 [   0    0    1    0  964    1    6    0    3    7]
 [   0    1    0    3    0  876    2    0    2    1]
 [   1    1    2    0    2    2  940    0    3    1]
 [   1    1    4    4    2    1    0 1010    2    2]
 [   1    1    6    3    1    2    1    4  955    2]
 [   0    0    0    2    9    2    0    3    2  988]]


In [24]:
#BestModel.save('Pretrained_model.h5')